In [ ]:
using MPSKit
using TensorKit
using KrylovKit
using TensorOperations
using MPSKitModels
function classical_Potts(q::Int64, β)
    V = ℂ^q
    A_potts = TensorMap(zeros,ComplexF64, V ⊗ V ← V ⊗ V)

    for i in 1:q
        for j in 1:q
            for k in 1:q
                for l in 1:q
                    E = -(Int(i == j) + Int(j == k) + Int(k == l) + Int(l == i))
                    A_potts[i, j, k, l] = exp(-β * E)
                end
            end
        end
    end
    return A_potts
end

function Potts_extended(q::Int64, β,K)
    V = ℂ^q
    A_potts = TensorMap(zeros,ComplexF64, V ⊗ V ⊗  V ⊗ V ←  V ⊗ V ⊗  V ⊗ V )

    for i in 1:q
        for j in 1:q
            for k in 1:q
                for l in 1:q
                    E = -(Int(i == j) + Int(j == k) + Int(k == l) + Int(l == i))
                    A_potts[i, j, k, l] = exp(-β * E)
                end
            end
        end
    end
    return A_potts
end


function Plaquette_Potts(q::Int64, β, J□;JΔ=0.0)
    V = ℂ^q
    data = ComplexF64[Int(i==j)*Int(j==k) for i=1:q, j=1:q, k=1:q]
    δ3 = TensorMap(data, V ← V ⊗ V)
    U = isometry(fuse(V, V), V ⊗ V)
    A_potts = TensorMap(zeros,ComplexF64, V ⊗ V ← V ⊗ V)
    
    for i = 1:q
        for j = 1:q
            for k = 1:q
                for l = 1:q
                    # 0.5 factor for double counting the weight on the bond
                    E = -(Int(i == j) + Int(j == k) + Int(k == l) + Int(l == i))/2
                    E -= J□ * Int(i == j == k == l)
                    E -= JΔ * (Int(i==j==k)+Int(j==k==l)+Int(k==l==i)+Int(l==i==j))
                    A_potts[i, j, k, l] = exp(-β * E)
                end
            end
        end
    end
    @tensor Afin[-1 -2;-3 -4] := A_potts[1 2;3 4]*δ3[5;1 11]*conj(δ3[2; 6 7])*conj(δ3[10;3 8])*δ3[4;9 12]*U[-1;5 6]*U[-2;7 8]*conj(U[-3;9 10])*conj(U[-4;11 12]);
    #return Afin
    return A_potts
end

Plaquette_Potts (generic function with 1 method)

In [15]:
N = 5
test_values_1 = zeros(ComplexF64,(2*N-1)^2)
l = length(test_values_1)
distx = 0.5## distance from alleged fixed point 0.079+0.060i in real
disty = 0.1# distance from alleged fixed point 0.079+0.060i in imaginary


## snake like structure of test_values will allow for faster convergence when recycling ψ (because you don't jump the entire distx after the loop)
for i in 1:1:(2*N-1)
    if div(i,2) == 1
        for j in 1:1:(2*N-1)
            if i <N+1
                if j<N+1
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+0.284 + 1im*LinRange(-disty,0.00,N)[j] .+0.0im
                else 
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+0.284 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.0im
                end
            else
                if j<N+1
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+0.284  + 1im*LinRange(-disty,0.00,N)[j]  .+0.0im
                else
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])   .+0.284 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.0im
                end
            end 
        end
    else
        for j in (2*N-1):-1:1
            if i <N+1
                if j<N+1
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+0.284 + 1im*LinRange(-disty,0.00,N)[j] .+0.0im
                else 
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+0.284 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.0im
                end
            else
                if j<N+1
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+ 0.284  + 1im*LinRange(-disty,0.00,N)[j]  .+0.0im
                else
                    test_values_1[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])   .+0.284 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.0im
                end
            end 
        end
    end
end

In [8]:
N = 5
test_values_2 = zeros(ComplexF64,(2*N-1)^2)
l = length(test_values_1)
distx = 0.5## distance from alleged fixed point 0.079+0.060i in real
disty = 0.3# distance from alleged fixed point 0.079+0.060i in imaginary


## snake like structure of test_values will allow for faster convergence when recycling ψ (because you don't jump the entire distx after the loop)
for i in 1:1:(2*N-1)
    if div(i,2) == 1
        for j in 1:1:(2*N-1)
            if i <N+1
                if j<N+1
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+1 + 1im*LinRange(-disty,0.00,N)[j] .+0.3im
                else 
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+1 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.3im
                end
            else
                if j<N+1
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+1  + 1im*LinRange(-disty,0.00,N)[j]  .+0.3im
                else
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])   .+1 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.3im
                end
            end 
        end
    else
        for j in (2*N-1):-1:1
            if i <N+1
                if j<N+1
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+1 + 1im*LinRange(-disty,0.00,N)[j] .+0.3im
                else 
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+1 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.3im
                end
            else
                if j<N+1
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+1  + 1im*LinRange(-disty,0.00,N)[j]  .+0.3im
                else
                    test_values_2[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])   .+1 + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+0.3im
                end
            end 
        end
    end
end

#### DMRG METHOD

In [3]:
## note that TNRKit is not compatible with MPSKIT -> add/rm respectively

function Potts_βc(q)
      return log(1+sqrt(q))
end

### Finite (leading boundary) (does not work)
# Results = zeros(ComplexF64,(3,l))
# L = 3
# ψ = FiniteMPS(L,ℂ^(Q^2), ℂ^D)
# # infinte (leading boudary works)
# for (i,J□) in enumerate(test_values)
#     plaq = (periodic_boundary_conditions(DenseMPO(Plaquette_Potts(Q,Potts_βc(Q),J□)),L))
#     # ## leading_boundary method 
#     ens = environments(ψ, plaq)
#     ψ, envs, = leading_boundary(ψ, plaq,  VUMPS(; verbosity=0,alg_eigsolve=MPSKit.Defaults.alg_eigsolve(;ishermitian=false)),ens)# use non-hermitian eigensolver
#     F = expectation_value(ψ, plaq)
#     S = first(entropy(ψ))
#     ξ = correlation_length(ψ)
#     Results[:,i] = [F,S,ξ]
#     println(i//l)
# end



# # ## INFITE (leading boundary)
# using MPSKit: InfiniteMPO
# Results = zeros(ComplexF64,(3,l))
# ψ = InfiniteMPS(ℂ^(Q^2), ℂ^D)
# # infinte (leading boudary works)
# for (i,J□) in enumerate(test_values)
#     plaq = qstate_clock() ### make sure boundary is correct

#     # ## leading_boundary method 
#     ens = environments(ψ, plaq)
#     ψ, envs, = leading_boundary(ψ, plaq,  VUMPS(; verbosity=0,alg_eigsolve=MPSKit.Defaults.alg_eigsolve(;ishermitian=false)),ens)# use non-hermitian eigensolver
#     F = expectation_value(ψ, plaq)
#     S = first(entropy(ψ))
#     ξ = correlation_length(ψ)
#     Results[:,i] = [F,S,ξ]
#     println(i//l)
# end








# DMRG method (finite)
    
#  #### block symmetric (have yet to try)
# ψ₀ = InfiniteMPS(ℂ^Q, ℂ^D);
# V = Vect[ZNIrrep{Q}](sector=>1 for sector in 0:Q-1)  
# # ψ₀ = FiniteMPS(L,V,Vect[ZNIrrep{Q}](sector=>D for sector in 0:Q-1)) 

using LoggingExtras
using MPSKit: updatetol, ∂∂AC,normalize!,add!, MPOHamiltonian,IterLog,loginit!,logiter!,logfinish!,logcancel!,fixedpoint, Algorithm, FiniteQP,LinearCombination,ProjectionOperator



### VUMPS ADAPTATION(doesnotconverge)
function excitations_alt(H, alg::QuasiparticleAnsatz, ϕ₀::FiniteQP,
    lenvs=environments(ϕ₀.left_gs, H),
    renvs=ϕ₀.trivial ? lenvs : environments(ϕ₀.right_gs, H); num=1)
qp_envs(ϕ) = environments(ϕ, H, lenvs, renvs)
E = effective_excitation_renormalization_energy(H, ϕ₀, lenvs, renvs)
H_eff(ϕ) = effective_excitation_hamiltonian(H, ϕ, qp_envs(ϕ), E)
Es, ϕs, convhist = eigsolve(H_eff, ϕ₀, num, :LM, alg.alg)

convhist.converged < num &&
@warn "excitation failed to converge: normres = $(convhist.normres)"

return Es, ϕs
end
function excitations_alt(H, alg::QuasiparticleAnsatz, lmps::FiniteMPS,
    lenvs=environments(lmps, H), rmps::FiniteMPS=lmps,
    renvs=lmps === rmps ? lenvs : environments(rmps, H);
    sector=one(sectortype(lmps)), num=1)
ϕ₀ = LeftGaugedQP(rand, lmps, rmps; sector)
return excitations(H, alg, ϕ₀, lenvs, renvs; num)
end

### DMRG ADAPTATION
@kwdef struct FiniteExcited_alt{A} <: Algorithm
    "optimization algorithm"
    gsalg::A = DMRG()
    "energy penalty for enforcing orthogonality with previous states"
    weight::Float64 = 1.0 ################THIS IS VARIABLE IN MY CODE -> because largest magn
end
function excitations_alt(H, alg::FiniteExcited_alt,
    states::Tuple{T,Vararg{T}},energies; #
    init=FiniteMPS([copy(first(states).AC[i])
                    for i in 1:length(first(states))]),
    num=1) where {T<:FiniteMPS}
num == 0 && return (scalartype(T)[], T[])
super_op = LinearCombination(tuple(H, ProjectionOperator.(states)...),
                energies)
envs = environments(init, super_op)
ne, _ = find_groundstate_alt(init, super_op, DMRG(maxiter = 500,tol=1e-6, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)), envs)
E = expectation_value(ne, H)
nstates = (states..., ne)
energies = (energies...,-E)
println(energies)
ens, excis = excitations_alt(H,alg, nstates,energies; init=init, num=num - 1) ## bit redudant, but don't want to screw up format

push!(ens, expectation_value(ne, H))
push!(excis, ne)

return ens, excis
end
function excitations_alt(H, alg::FiniteExcited_alt, ψ::FiniteMPS,energies; kwargs...) #,energies::Tuple{Float64}
return excitations_alt(H, alg, (ψ,),energies; kwargs...)
end






function calc_galerkin_alt(pos::Int, above::Union{InfiniteMPS,FiniteMPS,WindowMPS}, operator,
    below, envs)
AC´ = ∂∂AC(pos, above, operator, envs) * above.AC[pos]
normalize!(AC´)
out = add!(AC´, below.AL[pos] * below.AL[pos]' * AC´, -1)
return norm(out)
end

function find_groundstate_alt(ψ::FiniteMPS, H, alg, envs=environments(ψ, H))
    ϵs = map(pos -> calc_galerkin_alt(pos, ψ, H, ψ, envs), 1:length(ψ))
    ϵ = maximum(ϵs)
    log = IterLog("DMRG")

    LoggingExtras.withlevel(; alg.verbosity) do
        @infov 2 loginit!(log, ϵ, expectation_value(ψ, H, envs))
        for iter in 1:(alg.maxiter)
            alg_eigsolve = updatetol(alg.eigalg, iter, ϵ)

            zerovector!(ϵs)
            for pos in [1:(length(ψ) - 1); length(ψ):-1:2]
                h = ∂∂AC(pos, ψ, H, envs)
                _, vec = fixedpoint(h, ψ.AC[pos], :LM, alg_eigsolve)
                ϵs[pos] = max(ϵs[pos], calc_galerkin_alt(pos, ψ, H, ψ, envs))
                ψ.AC[pos] = vec
            end
            ϵ = maximum(ϵs)

            ψ, envs = alg.finalize(iter, ψ, H, envs)::Tuple{typeof(ψ),typeof(envs)}
            if ϵ <= 5e-5 ### I manually coded this because for some reason it would be turne dto 1e-10 for exications
                @infov 2 logfinish!(log, iter, ϵ, expectation_value(ψ, H, envs))
                break
            end
            if iter == alg.maxiter
                @warnv 1 logcancel!(log, iter, ϵ, expectation_value(ψ, H, envs))
            else
                @infov 3 logiter!(log, iter, ϵ, expectation_value(ψ, H, envs))
            end
        end
    end
    return ψ, envs, ϵ
end


find_groundstate_alt (generic function with 2 methods)

In [16]:
D=150
J=1
h=1
Q=5
using JLD2
l = length(test_values_1)
Results = zeros(ComplexF64,(2,l^2))

## fixing form first so others might converge quicker

# plaq = @mpoham sum(-1*Plaquette_Potts(Q,Potts_βc(Q),0.1){i,i+1} for i in vertices(FiniteChain(L))[1:end-1]) - Plaquette_Potts(Q,Potts_βc(Q),0.1){vertices(FiniteChain(L))[end],vertices(FiniteChain(L))[1]}  
# (ψ, envir , delta) = find_groundstate_alt(ψ, plaq, DMRG(maxiter = 500,tol=1e-4, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)))

for L in [4]
    l = length(test_values_1)
    Results = zeros(ComplexF64,(2,l,l))

    ψ = FiniteMPS(L,ℂ^(Q), ℂ^D)
    ψ2 = FiniteMPS(L,ℂ^(Q), ℂ^D)
    #ψ3 = FiniteMPS(L,ℂ^(Q), ℂ^D) if you want this need to change how init works
    for (i,J□) in enumerate(test_values_2)
        for (j,JΔ) in enumerate(test_values_1)
        println(i/l)
        plaq = @mpoham sum(-1*Plaquette_Potts(Q,Potts_βc(Q),J□,JΔ=JΔ){i,i+1} for i in vertices(FiniteChain(L))[1:end-1]) - Plaquette_Potts(Q,Potts_βc(Q),J□,JΔ=JΔ){vertices(FiniteChain(L))[end],vertices(FiniteChain(L))[1]}
        (ψ, envir , delta) = find_groundstate_alt(ψ, plaq, DMRG(maxiter = 500,tol=1e-5, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)))
        E = expectation_value(ψ, plaq)
        En, state  = excitations_alt(plaq, FiniteExcited_alt(gsalg = DMRG(eigalg =MPSKit.Defaults.alg_eigsolve(maxiter = 500,tol=1e-5,ishermitian=false))), (ψ,),(1.0,-E),init = ψ2,num=1) 
        #En, state = excitations_alt(plaq, QuasiparticleAnsatz(ishermitian=false), ψ, envir;num=1) #(no convergence)
        ψ2 = state[1]

        #S = [entropy(ψ,i) for i in 0:L] ###only gives the real parts (for now)
        println(E)
        println(En)
        #println(S)
        Results[1,i,j] = E
        Results[2,i,j] = En[1]
        #Results[3,i] = En[2]
        end
    end

    save_object("Big Run DMRG est Triangle $L,D$D",Results)
end

In [17]:
using PlotlyJS
using LaTeXStrings
using JLD2
L = 4
D = 250
Q = 5
function Potts_βc(q)
    return log(1+sqrt(q))
end
Results = load_object("Big Run DMRG est Triangle $L,D150")
for (j,J□) in enumerate(test_values_2)
    # z_values = (Results[1,:])/L
#     p = PlotlyJS.plot(PlotlyJS.contour(   z=real(z_values),
#     x=real(-im*test_values),
#         y=real(test_values),fill=true,colorbar=attr(
#             title="Re(F)", # title here
#             titleside="top",
#             titlefont=attr(
#                 size=14,
#                 family="Arial, sans-serif"
#             )
#         )),Layout(title=attr(text = "Free energy of finite Plaquette transfermatrix L=$L, D$D",x = 0.5),xaxis_title="Im(J□)",yaxis_title="Re(J□)"))


# PlotlyJS.savefig(p,"real free energy of finite PlaquetteL=$L, D$D.png") 


# z_values = (Results[1,:])/L
# p = PlotlyJS.plot(PlotlyJS.contour(   z=real(-1im*z_values),
#     x=real(-im*test_values),
#         y=real(test_values),fill=true,colorbar=attr(
#             title="Im(F)", # title here
#             titleside="top",
#             titlefont=attr(
#                 size=14,
#                 family="Arial, sans-serif"
#             )
#         )),Layout(title=attr(text = "Free energy of finite Plaquette transfermatrix L=$L, D$D",x = 0.5),xaxis_title="Im(J□)",yaxis_title="Re(J□)"))


# PlotlyJS.savefig(p,"Im free energy of finite PlaquetteL=$L, D$D.png") 

target = 0.134 − 0.021im
z_values = [log(abs((log(Results[1,j,i]/Results[2,j,i])  - ((2*pi)/L)*log(1+sqrt(Q))*target))) for i in 1:l]
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(z_values),
    x=real(-im*test_values_1),
        y=real(test_values_1),fill=true,colorbar=attr(
            title="log(|δ x_σ|)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = "Triangle DMRG |δ x_σ| J□ $J□ L=$L, D$D",x = 0.5),xaxis_title="Im(JΔ)",yaxis_title="Re(JΔ)"))


PlotlyJS.savefig(p,"scaling dimension Triangle J□ $J□ L=L=$L, D$D.png")
end

In [10]:
println(test_values)

In [12]:
L = 8
D = 50
Q = 5
using JLD2
using PlotlyJS
function Potts_βc(q)
    return log(1+sqrt(q))
end
Results = load_object("DMRG est Triangle 8,D50")

z_values = (Results[1,:])/L
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(z_values),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="Re(F)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = "Free energy of finite Plaquette transfermatrix L=$L, D$D",x = 0.5),xaxis_title="Im(J□)",yaxis_title="Re(J□)"))


PlotlyJS.savefig(p,"real free energy of finite PlaquetteL=$L, D$D.png") 


z_values = (Results[1,:])/L
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(-1im*z_values),
    x=real(-im*test_values_1),
        y=real(test_values_1),fill=true,colorbar=attr(
            title="Im(F)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = "Free energy of finite Plaquette transfermatrix L=$L, D$D",x = 0.5),xaxis_title="Im(J□)",yaxis_title="Re(J□)"))


PlotlyJS.savefig(p,"Im free energy of finite PlaquetteL=$L, D$D.png") 

target = 0.134 − 0.021im
z_values = [log(abs((log(Results[1,i]/Results[2,i])  - ((2*pi)/L)*log(1+sqrt(Q))*target))) for i in 1:l]
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(z_values),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="log(|δ x_σ|)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = "PTriangle DMRG |δ x_σ| L=$L, D$D",x = 0.5),xaxis_title="Im(J□)",yaxis_title="Re(J□)"))


PlotlyJS.savefig(p,"scaling dimension Triangle VUMPS L=$L, D$D.png")

SystemError: SystemError: opening file "DMRG est Triangle 8,D50": No such file or directory

In [7]:
using LinearAlgebra
function eye(m)
    return Matrix{ComplexF64}(I,m,m)
end

function Potts_βc(q)
    return log(1+sqrt(q))
end
function tmatrix(Q,beta,lambda)

    # H = zeros(ComplexF64,(Q^(2),Q^(2)))
    
    
  
    # H += lambda *reshape(sum(sum(    potts_phase_shift_combined(;q=Q,k=k,p=p).data for k in 1:1:Q-1) for p in 1:1:Q-1),(Q^2,Q^2))

    # H += -reshape(sum(potts_phase(;q=Q,k=k).data for k in 1:1:Q-1),(Q^2,Q^2))
 
    # H += -reshape(kron(eye(Q),sum((potts_spin_shift(; q = Q,k=j).data) for j in 1:1:Q-1)),(Q^2,Q^2)) ## i think it was the reshape!
    L = 2
    H = zeros(ComplexF64,(Q^L,Q^L))
    sigma = zeros(ComplexF64,Q,Q)
    tau = zeros(ComplexF64,Q,Q)
    identity_e = zeros(ComplexF64,Q,Q)
    for i in 1:Q
        sigma[i, i] = cis(2*pi*(i-1)/Q)
        tau[i,mod1(i + 1, Q)] = 1
        identity_e[i,i]= 1
    end
    for i in 1:L-1
        H +=  kron(kron(eye(Q^(i-1)),lambda* sum(sum(kron(tau^k, identity_e) * kron(sigma', sigma)^p + kron(identity_e ,tau^k) * kron(sigma', sigma)^p + kron(sigma', sigma)^k * kron(tau^p, identity_e) +  kron(sigma', sigma)^k * kron(identity_e ,tau^p) for k in 1:1:Q-1) for p in 1:1:Q-1)), eye(Q^(L-i-1)))
        H += -0.5*kron(kron(eye(Q^(i-1)),(sum((tau^k) for k in 1:1:Q-1))),eye(Q^(L-i))) 
        H +=  -kron(kron(eye(Q^(i-1)), sum(kron(sigma'^k,sigma^k) for k in 1:Q-1)), eye(Q^(L-i-1)))

    end
    H += -0.5*kron(eye(Q^(L-1)),sum((tau^k) for k in 1:1:Q-1)) #(not added since this is in the next one)
    
    Transfer_Matrix = exp(-log(1+sqrt(Q))*transpose(H) )   #### a little bit wrong since you work with beta/n -> in fractions this is okay but the first two plots are not representative
    V= ℂ^Q
    A_CCFT = TensorMap(Transfer_Matrix, V ⊗ V ← V ⊗ V)   
    return A_CCFT
end

tmatrix (generic function with 1 method)

In [8]:
D=30
J=1
h=1
Q=5
using JLD2
for L in [4] #,12,16,20,24
    l = length(test_values)
    Results = zeros(ComplexF64,(2,l))

    ψ = FiniteMPS(L,ℂ^(Q), ℂ^D)
    ψ2 = FiniteMPS(L,ℂ^(Q), ℂ^D)
    for (i,J□) in enumerate(test_values)
        println(i/l)
        plaq = @mpoham tmatrix(Q,Potts_βc(Q),J□){vertices(FiniteChain(L))[end],vertices(FiniteChain(L))[1]} + sum(tmatrix(Q,Potts_βc(Q),J□){i,i+1} for i in vertices(FiniteChain(L))[1:end-1]) 
        (ψ, envir , delta) = find_groundstate_alt(ψ, plaq, DMRG(maxiter = 500,tol=1e-5, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)))
        E = expectation_value(ψ, plaq)
        En, state  = excitations_alt(plaq, FiniteExcited_alt(gsalg = DMRG(eigalg =MPSKit.Defaults.alg_eigsolve(maxiter = 500,tol=1e-5,ishermitian=false))), (ψ,),(1.0,-E),init = ψ2,num=1) 
        #En0, st0 = excitations_alt(plaq, QuasiparticleAnsatz(ishermitian=false), ψ, envir;num=4) (no convergence)
        ψ2 = state[1]

        #S = [entropy(ψ,i) for i in 0:L] ###only gives the real parts (for now)
        println(E)
        println(En)
        #println(S)
        Results[1,i] = E
        Results[2,i] = En[1]
        #Results[3,i] = En[2]
    end

    save_object("Big Run DMRG CCFT est $L,D$D, potts",Results)
end

0.012345679012345678


┌ Info: DMRG init:	obj = +2.006993366413e+03 +2.378430484391e+03im	err = 2.6646e-01
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +2.391930961913e+03 +2.823111163091e+03im	err = 1.0012024307e-08	time = 6.14 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:157
┌ Info: DMRG conv 2:	obj = +2.391930962157e+03 +2.823111162964e+03im	err = 1.9123270791e-15	time = 6.76 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:151
┌ Info: DMRG init:	obj = -5.193801874567e+03 -6.120431622465e+03im	err = 7.5231e-01
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +1.618341912085e+03 +1.742233250087e+03im	err = 1.7916558653e-08	time = 1.36 se

(1.0, -2391.9309621573443 - 2823.111162964168im, -1618.3419118857134 - 1742.2332503837931im)
2391.9309621573443 + 2823.111162964168im
ComplexF64[1618.3419118857134 + 1742.2332503837931im]
0.024691358024691357


┌ Info: DMRG init:	obj = +1.926102416283e+03 +2.271559059135e+03im	err = 3.6438e-03
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +1.926247987483e+03 +2.271114458648e+03im	err = 3.0961374701e-10	time = 0.08 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:157
┌ Info: DMRG conv 2:	obj = +1.926247987482e+03 +2.271114458647e+03im	err = 1.6493474686e-15	time = 0.16 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:151
┌ Info: DMRG init:	obj = +1.324830020899e+03 +1.410293350308e+03im	err = 7.9932e-03
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +1.325644925339e+03 +1.407033715245e+03im	err = 1.7727260332e-09	time = 0.10 se

(1.0, -1926.247987481897 - 2271.1144586468577im, -1325.6449253303267 - 1407.0337152445989im)
1926.247987481897 + 2271.1144586468577im
ComplexF64[1325.6449253303267 + 1407.0337152445989im]
0.037037037037037035


...

┌ Info: DMRG conv 2:	obj = +4.354700905981e+02 -3.663083557701e+02im	err = 5.3944575596e-15	time = 0.44 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:151


526.8328274614521 - 609.227480247633im
ComplexF64[435.4700905980683 - 366.30835577009833im]
1.0


┌ Info: DMRG init:	obj = +4.235460703288e+02 -4.871004233281e+02im	err = 1.8242e-02
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +4.234339737597e+02 -4.848090850911e+02im	err = 1.3230624409e-08	time = 0.16 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:157
┌ Info: DMRG conv 2:	obj = +4.234339740488e+02 -4.848090852976e+02im	err = 6.1264475706e-15	time = 0.29 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:151
┌ Info: DMRG init:	obj = +3.725691849954e+02 -2.948004757982e+02im	err = 3.1460e-02
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:136
┌ Info: DMRG   1:	obj = +3.684215963380e+02 -2.777263884388e+02im	err = 4.1643241837e-08	time = 0.25 se

(1.0, -423.4339740487866 + 484.8090852975947im, -368.42159248269127 + 277.7263875704977im)
423.4339740487866 - 484.8090852975947im
ComplexF64[368.42159248269127 - 277.7263875704977im]


┌ Info: DMRG conv 2:	obj = +3.684215924827e+02 -2.777263875705e+02im	err = 1.8048643957e-14	time = 0.46 sec
└ @ Main c:\Users\vicva\OneDrive\Documenten\git\Thesis_CCFT\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:151


In [3]:

function plot_difference(results,z_values,dif,test_values,Q,L,D)
   
    p = PlotlyJS.plot(PlotlyJS.contour(   z=real(z_values),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="Re(F)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = " LM DMRG Fr2ee energy of CCFT transfermatrix L=$L,Q=$Q",x = 0.5),xaxis_title="Im(lambda)",yaxis_title="Re(lambda)"))


PlotlyJS.savefig(p,"Q$Q   LM DMRG fig 1 Free energy of CCFT L=$L.png") 
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(-1im*z_values),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="Im(F)", # title here
            titleside="top",
            titlefont=attr(
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = "Im  LM DMRG free energy of CCFT transfermatrix L=$L,Q=$Q D=$D",x = 0.5),xaxis_title="Im(lambda)",yaxis_title="Re(lambda)"))
PlotlyJS.savefig(p,"Q$Q  LM DMRG fig 2 Im Free energy of CCFT L=$L.png") 
p = PlotlyJS.plot(PlotlyJS.contour(   z=dif,
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="|δ x_σ|", # title here
            titleside="top",
            titlefont=attr( 
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = " LM DMRG CCFT|δ x_σ| L=$L,Q=$Q,D=$D",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)"))
PlotlyJS.savefig(p,"Q$Q fig 3 LM DMRG  CCFT scaling energy of CCFT L=$L D=$D.png") 
pp = PlotlyJS.plot(PlotlyJS.contour(   z=(real(1 ./ results)),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="log(ξ)", # title here
            titleside="top",
            titlefont=attr( 
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = " LM DMRG (C)CFT ξ L=$L,Q=$Q D=$D",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)"))
PlotlyJS.savefig(pp,"Q$Q fig 4 LM DMRG CCFT ξ of L=$L D=$D.png") 
end

plot_difference (generic function with 1 method)

In [ ]:
using JLD2
using PlotlyJS
Q =5
L = 4
D = 30
first_two = load_object("Big Run DMRG CCFT est $L,D$D, potts")
println(size(first_two[1]))
Results = zeros(ComplexF64,(l))
target = 0.134 - 0.021im
# target = 2/15
dif = zeros(Float64,(l))
z_values = zeros(ComplexF64,(l))    
for j in 1:l
    Results[j] = log(((abs(first_two[1,j])/abs(first_two[2,j]))))
    z_values[j] = 1/log(1+sqrt(Q)) * log((abs(first_two[2,j])))/L

    dif[j] =log(abs((log(first_two[1,j]/first_two[2,j])  - ((2*pi)/L)*log(1+sqrt(Q))*target))) #log(abs(log(eig[1]/eig[2]) - log(1+sqrt(Q))*L/(2*pi)*target))
    
end
plot_difference(Results,z_values,dif,test_values,Q,L,D)

()
[-1.549042126794479, -1.579580745996088, -1.6091581186722066, -1.634380140126764, -1.6490982012690285, -1.6431685842922037, -1.602048762335164, -1.5087234516892774, -1.3462210176273206, -1.5915021112679775, -1.633802133585174, -1.6792684604575792, -1.7256597746228652, -1.767307267478493, -1.7919635841439872, -1.7774520174635715, -1.6936411597856917, -1.514747534997677, -1.6300465738230454, -1.684616537097374, -1.7479239156190622, -1.8206169776005943, -1.900732954107843, -1.9781840057956848, -2.0228988515031587, -1.9766154573980994, -1.7919694487605056, -1.6611404282608146, -1.7266202994968132, -1.806845755936493, -1.9069998476705168, -2.033650650945521, -2.191090524441496, -2.3587955424480076, -2.415491976867339, -2.1879864743002044, -1.68137451953093, -1.7541300013080863, -1.8461280424589808, -1.9668522302610107, -2.1332915783605912, -2.378677835439157, -2.7695398251570027, -3.243586760202281, -2.7319590423161477, -1.6883899505735838, -1.762979856445796, -1.8578156941643633, -1.983

"Q5 fig 4 LM DMRG CCFT ξ of L=4 D=30.png"

In [ ]:
L = 4
D = 30
first_two = load_object("Big Run DMRG CCFT est $L,D$D, potts")
println(size(first_two[1]))
Results = zeros(ComplexF64,(l))
target = 0.134 - 0.021im
# target = 2/15
dif = zeros(Float64,(l))
z_values = zeros(ComplexF64,(l))    
for j in 1:l
    Results[j] = log(((abs(first_two[1,j])/abs(first_two[2,j]))))
    z_values[j] = 1/log(1+sqrt(Q)) * log((abs(first_two[2,j])))/L

    dif[j] =log(abs((log(first_two[1,j]/first_two[2,j])  - ((2*pi)/L)*log(1+sqrt(Q))*target))) #log(abs(log(eig[1]/eig[2]) - log(1+sqrt(Q))*L/(2*pi)*target))
    
end
plot_difference(Results,z_values,dif,test_values,Q,L,D)

# Leading Boundary

In [ ]:
l = length(test_values)
Results = zeros(ComplexF64,(3,l))
Q = 5
D = 25
ψ = InfiniteMPS(ℂ^(Q^2), ℂ^D)

#infinte(leading boudary works)
for (i,lambda) in enumerate(test_values)
    plaq = InfiniteMPO([-1*Plaquette_Potts(Q,Potts_βc(Q),lambda),]) ### make sure boundary is correct

    # ## leading_boundary method 
    ens = environments(ψ, plaq)
    ψ, envs, = leading_boundary(ψ, plaq,  VUMPS(; verbosity=0,alg_eigsolve=MPSKit.Defaults.alg_eigsolve(;ishermitian=false)),ens)# use non-hermitian eigensolver
    F = expectation_value(ψ, plaq)
    S = first(entropy(ψ))
    ξ = correlation_length(ψ)
    Results[:,i] = [F,S,ξ]
    println(i//l)
end
using JLD2
save_object("Leading_boundary Plasuette D$D",Results)

1//81


In [5]:
l = length(test_values)
Results = zeros(ComplexF64,(3,l))
Q = 5
D = 25
ψ = InfiniteMPS(ℂ^(Q), ℂ^D)

#infinte(leading boudary works)
for (i,lambda) in enumerate(test_values)
    plaq = InfiniteMPO([-1*tmatrix(Q,Potts_βc(Q),lambda),]) ### make sure boundary is correct

    # ## leading_boundary method 
    ens = environments(ψ, plaq)
    ψ, envs, = leading_boundary(ψ, plaq,  VUMPS(; verbosity=0,alg_eigsolve=MPSKit.Defaults.alg_eigsolve(;ishermitian=false)),ens)# use non-hermitian eigensolver
    F = expectation_value(ψ, plaq)
    S = first(entropy(ψ))
    ξ = correlation_length(ψ)
    Results[:,i] = [F,S,ξ]
    println(i//l)
end
using JLD2
save_object("Leading_boundary CCFT D$D",Results)

1//81
2//81
1//27
4//81
5//81
2//27
7//81
8//81
1//9
10//81
11//81
4//27
13//81
14//81
5//27
16//81
17//81
2//9
19//81
20//81
7//27
22//81
23//81
8//27
25//81
26//81
1//3
28//81
29//81
10//27
31//81
32//81
11//27
34//81
35//81
4//9
37//81
38//81
13//27
40//81
41//81
14//27
43//81
44//81
5//9
46//81
47//81
16//27
49//81
50//81
17//27
52//81
53//81
2//3
55//81
56//81
19//27
58//81
59//81
20//27
61//81
62//81
7//9
64//81
65//81
22//27
67//81
68//81
23//27
70//81
71//81
8//9
73//81
74//81
25//27
76//81
77//81
26//27
79//81
80//81
1//1


In [ ]:
using PlotlyJS
Results = load_object("Leading_boundary Plasuette D25")
println(size(Results))
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(Results[1,:]),
x=real(-im*test_values),
    y=real(test_values),fill=true,colorbar=attr(
        title="Re(F)", # title here
        titleside="top",
        titlefont=attr(
            size=14,
            family="Arial, sans-serif"
        )
    )),Layout(title=attr(text = " Leading boundary Free energy of Plaquette transfermatrix Q=$Q,D$D",x = 0.5),xaxis_title="Im(lambda)",yaxis_title="Re(lambda)"))


PlotlyJS.savefig(p,"Q$Q  Leading boundary fig 1 Free energy of Plaquette$D.png") 
p = PlotlyJS.plot(PlotlyJS.contour(   z=real(-1im*Results[1,:]),
x=real(-im*test_values),
    y=real(test_values),fill=true,colorbar=attr(
        title="Im(F)", # title here
        titleside="top",
        titlefont=attr(
            size=14,
            family="Arial, sans-serif"
        )
    )),Layout(title=attr(text = "Im Leading boundary free energy of Plaquettetransfermatrix Q=$Q,D$D",x = 0.5),xaxis_title="Im(lambda)",yaxis_title="Re(lambda)"))
PlotlyJS.savefig(p,"Q$Q Leading boundary  fig 2 Im Free energy of PlaquetteD$D.png") 
# p = PlotlyJS.plot(PlotlyJS.contour(   z=,
#     x=real(-im*test_values),
#         y=real(test_values),fill=true,colorbar=attr(
#             title="|δ x_σ|", # title here
#             titleside="top",
#             titlefont=attr( 
#                 size=14,
#                 family="Arial, sans-serif"
#             )
#         )),Layout(title=attr(text = " LM DMRG (C)CFT|δ x_σ| L=$L,Q=$Q",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)"))
# PlotlyJS.savefig(p,"Q$Q  fig 3  LM DMRG scaling energy of CCFT L=$L,.png") 
pp = PlotlyJS.plot(PlotlyJS.contour(   z=real(Results[3,:]),
    x=real(-im*test_values),
        y=real(test_values),fill=true,colorbar=attr(
            title="log(ξ)", # title here
            titleside="top",
            titlefont=attr( 
                size=14,
                family="Arial, sans-serif"
            )
        )),Layout(title=attr(text = " Leading boundary Plaquette ξ ,Q=$Q,$D",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)"))
PlotlyJS.savefig(pp,"Q$Q fig 4 Leading boundary Plaquette ξD$D.png") 


(3, 81)


"Q5 fig 4 Leading boundary Plaquette ξD25.png"